In [38]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D,Dense, Flatten, GlobalAveragePooling2D, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.python.keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from os.path import join, exists, expanduser
from os import listdir, makedirs
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
import keras as k

['train', 'labels.csv', 'sample_submission.csv', 'test']


In [39]:
#training_set = train_datagen.flow_from_directory('../input/train', target_size = (224, 224, 1),batch_size = 32)
#test_set = test_datagen.flow_from_directory('../input/test', target_size = (224, 224, 1),batch_size = 32)

data_dir = '../input'
labels = pd.read_csv(join(data_dir, 'labels.csv'))
Test_submission=pd.read_csv(join(data_dir, 'sample_submission.csv'))
#train_dir=train_datagen.flow_from_directory('../input/train', target_size = (224, 224, 1),batch_size = 32)
#test=test_datagen.flow_from_directory('../input/test', target_size = (224, 224, 1),batch_size = 32)

In [40]:
Test_submission.head(5)



,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333

In [41]:
Test_submission.shape

(10357, 121)

In [42]:
labels.head(5)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [43]:
labels.breed.head(5)

0         boston_bull
1               dingo
2            pekinese
3            bluetick
4    golden_retriever
Name: breed, dtype: object

In [44]:
labels.pivot_table(index='breed',aggfunc=len).sort_values('id',ascending=False)



,id
breed,
scottish_deerhound,126
maltese_dog,117
afghan_hound,116
entlebucher,115
bernese_mountain_dog,114
shih-tzu,112
great_pyrenees,111
pomeranian,111
basenji,110


In [45]:
im_size = 224

x_train = []
y_train = []

i = 0 
for f, breed in tqdm(labels.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = labels.breed[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1


100%|██████████| 10222/10222 [00:42<00:00, 240.11it/s]


In [46]:
x_test = []

for f in tqdm(Test_submission['id'].values):
    img = cv2.imread('../input/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [00:42<00:00, 246.57it/s]


In [47]:
#y_train=np.array(y_train)
x_train_new=np.array(x_train)

x_test_new=np.array(x_test)

In [48]:
x_train_new.shape,x_test_new.shape

((10222, 224, 224, 3), (10357, 224, 224, 3))

In [49]:
num_class=120
le=preprocessing.LabelEncoder()
Y=le.fit_transform(y_train)
Y=k.utils.to_categorical(Y, num_class)

In [50]:
Y.shape

(10222, 120)

In [51]:

#X_train, X_test, Y_test, Y_valid = train_test_split(x_train, Y, test_size=0.3, random_state=1)


In [52]:
#resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [53]:
pretrained_model = ResNet50(weights='imagenet')

In [ ]:
print('Output_layer_type= {}'.format(pretrained_model.layers[-1]))
print('Output_layer_shape= {}'.format(pretrained_model.layers[-1].output_shape))


In [ ]:
from keras.utils.vis_utils import plot_model
from IPython.display import Image


In [ ]:
plot_model(pretrained_model, show_shapes=True, to_file='pretrained_model.png')
Image(retina=True, filename='pretrained_model.png')

In [ ]:
pretrained_model.layers.pop()

In [ ]:
print('Output_layer_type= {}'.format(pretrained_model.layers[-1]))
print('Output_layer_shape= {}'.format(pretrained_model.layers[-1].output_shape))

In [ ]:
for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
model = k.models.Sequential()
model.add(pretrained_model)

model.add(k.layers.Dense(num_class, activation='softmax'))
model.summary()

In [ ]:

model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
callbacks_list = [k.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()


In [ ]:
print('Input Shape = {}'.format(model.layers[0].input_shape))
print('Shape Shape = {}'.format(model.layers[-1].output_shape))

In [ ]:
model_log=model.fit(
    x=x_train_new, y=Y, batch_size=32,
    epochs=20, verbose=1,validation_split=.2
     
)

In [ ]:
# plotting the metrics
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(model_log.history['acc'])
#plt.plot(model_log.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

In [ ]:
#model.save("model.h5")

#model.save_weights('model_weight.h5')

In [ ]:
prediction=model.predict(x_test_new, verbose=1)
#prediction

In [ ]:
sub = pd.DataFrame(prediction)



In [ ]:
sub.shape


In [ ]:
sub.shape,x_test_new.shape

In [ ]:
Test_submission1=Test_submission

In [ ]:
# Set column names to those generated by the one-hot encoding earlier

# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', Test_submission1['id'])
sub.head(5)

In [ ]:
col_names = Test_submission1.columns.values
sub.columns = col_names

In [ ]:
sub.head(5)

In [ ]:
sub.to_csv('submission1.csv', index = False)